In [14]:

import os.path
import pickle

from Ballot import Ballot
from CandidateModel import CandidateModel
from DefaultConfigOptions import *
from ElectionConstructor import ElectionConstructor, construct_irv, construct_h2h
from ModelStats import ModelStats
from NDPopulation import NDPopulation
from ProcessResult import ProcessResult
from Timings import Timings


In [15]:
def create_model_and_population(ideology_bins: int, ideology_dim: int) -> (CandidateModel, NDPopulation):
    ideology_bins = 64
    hidden_ratio = 4
    n_hidden = hidden_ratio * ideology_bins * ideology_dim
    n_latent = ideology_bins * ideology_dim
    batch_size = 128
    learn_rate = .001

    model = CandidateModel(ideology_bins=ideology_bins,
                                     ideology_dim=ideology_dim,
                                     n_hidden=n_hidden,
                                     n_latent = n_latent,
                                     learn_rate= learn_rate)

    population_means = np.zeros(shape=(ideology_dim,))
    population_stddev = np.ones(shape=(ideology_dim,))
    pop = NDPopulation(population_means, population_stddev)
    return model, pop

In [ ]:
def measure_representation(candidate: Candidate, voters: List[Voter]) -> float:
    n_voters = len(voters)
    balance = []
    for d in candidate.ideology.dim:
        lc = len([v for v in voters if v.ideology.vec[d] < candidate.ideology.vec[d]])
        balance.append(min(lc / n_voters, 1 - lc / n_voters))
    return float(np.mean(balance))

In [16]:
def run_sample_election(model: CandidateModel, process: ElectionConstructor, population: NDPopulation, timings: Timings):
    candidates = []
    model_entries = set(np.random.choice(range(6), 3, replace=False))
    for i in range(6):
        if i in model_entries and model.ready():
            ideology = Ideology(model.choose_ideology(candidates))
            c = Candidate("m-" + str(i), Independents, ideology, 0)
        else:
            ideology = population.unit_sample_voter().ideology
            c = Candidate("r-" + str(i), Independents, ideology, 0)

        candidates += [c]

    voters = population.generate_unit_voters(1000)
    ballots = [Ballot(v, candidates, unit_election_config) for v in voters]
    result = process.run(ballots, set(candidates))
    winner = result.winner()
    balance = measure_representation(winner, voters)

    return winner, candidates, balance


In [18]:


def train_candidate_model(model: CandidateModel, process: ElectionConstructor, population: NDPopulation):
    timings = Timings()
    stats = ModelStats()
    first = True
    while model.global_step < 5000:
        with timings.time_block("run_election"):
            winner, candidates = run_sample_election(model, process, population, timings)
        with timings.time_block("add_sample"):
            for i in range(len(candidates)):
                model.add_sample_from_candidates(candidates[i], candidates[0:i], winner)

        if model.ready():
            if first:
                print("starting to train")
                first = False

            stats.update(winner, candidates)
            with timings.time_block("model.train"):
                model.train(128)
            s = model.global_step
            if (s < 100 and s % 10 == 0) or (s < 1000 and s % 100 == 0) or s % 1000 == 0:
                stats.print(process.name, model.global_step)
                if model.global_step < 10000:
                    stats.reset()

    timings.print()

In [19]:
def check_stats(stats: ModelStats, model: CandidateModel, process: ElectionConstructor, population: NDPopulation):
    results=[]
    timings = Timings()
    for i in range(1000):
        winner, candidates, balance = run_sample_election(model, process, population, timings)
        stats.update(winner, candidates, balance)

In [20]:
def run_parameter_set(process: ElectionConstructor, ibins: int, dim: int):
    save_path = "models/cm-%s-%03d-%dD.p" % (process.name, ibins, dim)
    model, population = create_model_and_population(ibins, dim)
    if os.path.exists(save_path):
        with open(save_path, "rb") as f:
            model: CandidateModel = pickle.load(f)
    else:
        train_candidate_model(model, process, population)
        # Saving the model file is not working at this time.
        # model.save_to_file(save_path)

    stats = ModelStats()
    check_stats(stats, model, process, population)
    return stats, model

In [21]:
dims = [4]
processes = [
    ElectionConstructor(constructor=construct_irv, name="Instant Runoff"),
    ElectionConstructor(constructor=construct_h2h, name="Head-to-Head")
]

results = []
for bins in [64, 128]:
    for process in processes:
        for dim in dims:
            stats, model = run_parameter_set(process, bins, dim)
            results.append(ProcessResult(process, bins, dim, stats))
            results[-1].print()

for r in results:
    r.print()

starting to train
 Instant Runoff     10,    10 random     33/     3  9.09% O:  0.76 model     27/     7 25.93% O:  0.76 chance of model_winner = 70.00%
 Instant Runoff     20,    10 random     30/     1  3.33% O:  0.89 model     30/     9 30.00% O:  0.48 chance of model_winner = 90.00%
 Instant Runoff     30,    10 random     30/     0  0.00% O:  0.00 model     30/    10 33.33% O:  0.47 chance of model_winner = 100.00%
 Instant Runoff     40,    10 random     30/     0  0.00% O:  0.00 model     30/    10 33.33% O:  0.34 chance of model_winner = 100.00%
 Instant Runoff     50,    10 random     30/     0  0.00% O:  0.00 model     30/    10 33.33% O:  0.16 chance of model_winner = 100.00%
 Instant Runoff     60,    10 random     30/     0  0.00% O:  0.00 model     30/    10 33.33% O:  0.16 chance of model_winner = 100.00%
 Instant Runoff     70,    10 random     30/     0  0.00% O:  0.00 model     30/    10 33.33% O:  0.20 chance of model_winner = 100.00%
 Instant Runoff     80,    10 ra

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tensorflow as tf
x = np.array([x for x in range(100)])
tf.gather(x, [10,20,30,40])